In [1]:
import pandas as pd
import numpy as np 


Preparing Data

In [2]:
movies = pd.read_csv('../Linear_regression/data_set/Movies/movies.csv')
ratings = pd.read_csv('../Linear_regression/data_set/Movies/ratings.csv')

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


Data Scraping

In [4]:
#Using regular expresion to delete year from tittle
movies['year'] = movies['title'].str.extract('(\(\d\d\d\d\))',expand = False)
movies['year'] = movies['year'].str.extract('(\d\d\d\d)',expand=False)
movies['title'] = movies['title'].str.replace('(\(\d\d\d\d\))', '')
movies['title'] = movies['title'].apply(lambda x: x.strip())
movies['genres'] = movies['genres'].str.split(pat='|')
movies.head()

C:\Users\shina\AppData\Local\Temp\ipykernel_14040\3386939449.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  movies['title'] = movies['title'].str.replace('(\(\d\d\d\d\))', '')


,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [5]:
movies_hot_encode = pd.concat([             #Hot encode wykorzysytując funkcję biblioteki pandas concat oraz funkcji applay
        movies.drop("genres", axis = 1),
        movies.genres.apply(lambda x: pd.Series(1, x)).fillna(0)
    ], axis=1, )
movies_hot_encode.drop(["title","year","(no genres listed)"],axis=1,inplace=True)
movies_hot_encode.set_index(keys='movieId',drop=True,inplace=True)

In [6]:
movies_hot_encode.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir
movieId,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


User data

In [7]:
user_movies = ratings.merge(movies, left_on='movieId',right_on='movieId')#połączenie ze sobą dwóch tabel danych
user_movies.drop(['title','timestamp','genres','year'],inplace=True,axis = 1)
user_movies.set_index(keys='movieId',drop=True,inplace=True)

Recommender System

In [16]:
x = 5
idx  = user_movies.loc[user_movies['userId'] == x,:].index.tolist()#utworzenie wektora z normalizowanego do doboru polecenia filmu
value_list = movies_hot_encode.loc[idx,:]
user_ratings = user_movies.loc[user_movies['userId'] == x,'rating'].T
genre_ratings = user_ratings.dot(value_list)
genre_ratings_normalize = (genre_ratings-genre_ratings.min())/(genre_ratings.max()-genre_ratings.min())


In [20]:
type(user_ratings)

pandas.core.series.Series

In [9]:
idx1  = user_movies.loc[user_movies['userId'] != x,:].index.tolist() # obliczenie średniej filmów
un_seen_films = movies_hot_encode.loc[np.unique(idx1),:]#użyte np.unique zamiast pd.drop_duplicates ponieważ funkcja z pandasa usuwała wartości orginalne   
ratings_un_seen_films = genre_ratings_normalize*un_seen_films
ratings_un_seen_films['Avarege']=ratings_un_seen_films.sum(axis=1)


Expected value

In [10]:
top20 = ratings_un_seen_films.loc[:,'Avarege'].sort_values(ascending = False).iloc[0:20]#wybór 20 najlepszych filmów do polecenia 
movies.loc[movies['movieId'].isin(top20.index),:]

,movieId,title,genres,year
19,20,Money Train,"[Action, Comedy, Crime, Drama, Thriller]",1995
118,145,Bad Boys,"[Action, Comedy, Crime, Drama, Thriller]",1995
400,459,"Getaway, The","[Action, Adventure, Crime, Drama, Romance, Thr...",1994
743,970,Beat the Devil,"[Adventure, Comedy, Crime, Drama, Romance]",1953
1390,1907,Mulan,"[Adventure, Animation, Children, Comedy, Drama...",1998
1394,1912,Out of Sight,"[Comedy, Crime, Drama, Romance, Thriller]",1998
2903,3893,Nurse Betty,"[Comedy, Crime, Drama, Romance, Thriller]",2000
3460,4719,Osmosis Jones,"[Action, Animation, Comedy, Crime, Drama, Roma...",2001
3608,4956,"Stunt Man, The","[Action, Adventure, Comedy, Drama, Romance, Th...",1980
3657,5027,Another 48 Hrs.,"[Action, Comedy, Crime, Drama, Thriller]",1990
